In [ ]:
import json 
from dataclasses import dataclass

from databricks.sdk import WorkspaceClient
from databricks.labs.lakebridge import __version__
from databricks.labs.lakebridge.config import (
    DatabaseConfig,
    ReconcileConfig,
    ReconcileMetadataConfig
)
from databricks.labs.lakebridge.reconcile.recon_config import Table
from databricks.labs.lakebridge.reconcile.trigger_recon_service import TriggerReconService

In [ ]:
dbutils.widgets.text("recon_config", "")
dbutils.widgets.text("target_env", "dev")

table_configs = json.loads(dbutils.widgets.get("recon_config"))
env = dbutils.widgets.get("target_env")

print(f"Loaded {len(table_configs)} table configs for env = {env}")

In [ ]:
ws = WorkspaceClient(
    product="lakebridge",
    product_version=__version__
)

In [ ]:
@dataclass
class TableRecon:
    source_schema: str
    target_catalog: str
    target_schema: str
    tables: list
    source_catalog: str | None = None

In [ ]:
results_schema = f"lakebridge_recon_{env}"

In [ ]:

# Trigger Reconciliation per Table


for idx, cfg in enumerate(table_configs, start=1):
    try:
        source_full = f"{cfg['source_catalog']}.{cfg['source_schema']}.{cfg['source_table']}"
        target_full = f"{cfg['target_catalog']}.{cfg['target_schema']}.{cfg['target_table']}"

        print(f"\n[{idx}/{len(table_configs)}] Reconciling {source_full} -> {target_full}")

        
        # Reconcile Config (CSV driven)
        
        reconcile_config = ReconcileConfig(
            data_source=cfg["data_source"],                 
            report_type=cfg["report_type"].lower(),         
            secret_scope=cfg["secret_scope"],

            database_config=DatabaseConfig(
                source_catalog=cfg["source_catalog"],
                source_schema=cfg["source_schema"],
                target_catalog=cfg["target_catalog"],
                target_schema=cfg["target_schema"]
            ),

            # IMPORTANT: write results into target catalog only
            metadata_config=ReconcileMetadataConfig(
                catalog=cfg["target_catalog"],
                schema=results_schema
            )
        )

        
        # Table Config
        
        table_recon = TableRecon(
            source_schema=cfg["source_schema"],
            target_catalog=cfg["target_catalog"],
            target_schema=cfg["target_schema"],
            source_catalog=cfg["source_catalog"],
            tables=[
                Table(
                    source_name=cfg["source_table"],
                    target_name=cfg["target_table"],
                    join_columns=cfg["join_columns"]
                )
            ]
        )

        
        # Trigger Reconciliation
        
        result = TriggerReconService.trigger_recon(
            ws=ws,
            spark=spark,
            table_recon=table_recon,
            reconcile_config=reconcile_config
        )

        print(f"SUCCESS | {cfg['target_table']} | Recon ID: {result.recon_id}")

    except Exception as e:
        print(f"FAILED | {cfg['target_table']} | Error: {str(e)}")
